# Simple 3-node model

This model demonstrates how to create different types of component and link them together. We use a built-in plugboard component to load timeseries data from a CSV file. A second node computes a rolling sum of these values. Finally another built-in component saves the output to a different CSV.

In [ ]:
import typing as _t

from plugboard.connector import AsyncioChannel, Connector
from plugboard.component import Component
from plugboard.component import IOController as IO
from plugboard.process import Process
from plugboard.schemas import ConnectorSpec
from plugboard.contrib.components import CSVReader, CSVWriter

The `CSVReader` component is provided by plugboard. It takes the content of a CSV and emits the values row-by-row.

In [ ]:
input_data = CSVReader("input.csv")

Next, we implement a component to compute a running total using its internal state.

In [ ]:
class RunningTotal(Component):
    io = IO(inputs=["value"], outputs=["total_value"])

    def __init__(self, iters: int, *args: _t.Any, **kwargs: _t.Any) -> None:
        super().__init__(*args, **kwargs)
        self._running_total = 0

    async def step(self) -> None:
        self._running_total += self.value
        self.total_value = self._running_total

In [ ]:
total = RunningTotal()

For the output we can use the built-in `CSVWriter` component.

In [ ]:
ouput_data = CSVWriter("output.csv")

Now connect the components together in a `Process`.

In [ ]:
process = Process(
    components=[input_data, total, ouput_data],
    connectors=[
        # Connect input_data to total
        Connector(spec=ConnectorSpec(source="input_data.reading", target="total.value"), channel=AsyncioChannel()),
        # Connect total to output_data
        Connector(spec=ConnectorSpec(source="total.total_value", target="output_data.output_value"), channel=AsyncioChannel()),
    ],
)

Now we can initialise and run the simulation.

In [ ]:
await process.init()
await process.run()

Finally check we have the output data saved in `output.csv`.

In [ ]:
!cat output.csv